In [17]:
import bingmaps
from bingmaps.apiservices import LocationByQuery
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from pandarallel import pandarallel


In [ ]:
%pip install bingmaps

In [ ]:
%pip install pandarallel

In [4]:
df_location = pd.read_excel('C:/Users/ihsankoo/Desktop/Clarusway/Data Science/Car price prediction capstone project/location.xlsx')

In [18]:
bingmaps_key = pd.read_text('C:/Users/ihsankoo/Desktop/Clarusway/Data Science/Car price prediction capstone project/bingmaps_key.txt')
def getCorrectedAddressAndCoordinates(x):
    try:
        address = x

        # set up the query parameters
        params = {
            'q': address,
            'key': bingmaps_key,
        }

        # use the geocode API to get the corrected address
        response = LocationByQuery(params)

        # print the corrected address
        if response.status_code == 200:
            return response.get_address[0]["formattedAddress"],response.get_coordinates[0][0], response.get_coordinates[0][1]
        else:
            return address,np.nan,np.nan
    except:
        return x,np.nan,np.nan
getCorrectedAddressAndCoordinates(df_location['location'][5])

('Beethovenstr 89, 42655 Solingen, Germany', 51.176936, 7.06473)

In [20]:
df_location['location_corrected'] = df_location.location.apply(lambda x: getCorrectedAddressAndCoordinates(x))

In [22]:
df_location['address'] = df_location.location_corrected.apply(lambda x: x[0])
df_location['latitude'] = df_location.location_corrected.apply(lambda x: x[1])
df_location['longtitude'] = df_location.location_corrected.apply(lambda x: x[2])
df_location.drop(columns=['location_corrected'], inplace = True)

In [23]:
df_location.address

0        Calle Forja 6, 11500 El Puerto de Santa María,...
1        Calle Logroño, 28649 Rozas de Puerto Real, Mad...
2                Muelle de Llevant, 43004 Tarragona, Spain
3        Calle Provenza, Valle de los Molinos, 45200 Gu...
4            Carril de los Aragonés 4, 30007 Murcia, Spain
                               ...                        
28625    Calle de los Hermanos Lumière 16, 29004 Málaga...
28626            Stobbeweg 6, 2461 EX Ter Aar, Netherlands
28627                                        5060, Belgium
28628    Bergambachterstraat 5, 2871 JB Schoonhoven, Ne...
28629            Lage Ham 106, 5102 AE Dongen, Netherlands
Name: address, Length: 28630, dtype: object

In [34]:
df_location.sample(5)

,index,location,address,latitude,longitude
5542,5542,"49086 Osnabrück, DE","49086, NI, Germany",52.247692,8.109242
2191,2191,"Wilhelm-Rausch-Straße 11, 31228 Peine, DE","Wilhelm-Rausch Straße 11, 31228 Peine, Germany",52.340208,10.243982
27073,27073,"CALLE FRANCISCO ALONSO HIDALGO MARTINEZ, Nº62...","Calle Francisco Alfonso Hidalgo Martínez, 3010...",38.027716,-1.169542
2091,2091,"Neuköllner Straße 234, 12357 Berlin, DE","Neuköllner Straße 234, 12357 Berlin, Germany",52.427250,13.480350
28582,28582,"Liebstädter Str. 5, 01277 Dresden, DE","Liebstädter Str 5, 01277 Dresden, Germany",51.026935,13.784585


In [25]:
df_location.drop(columns=['Unnamed: 0'], inplace = True)

In [33]:
df_location.rename(columns={'longtitude':'longitude'}, inplace = True)

In [27]:
df_location.to_csv('C:/Users/ihsankoo/Desktop/Clarusway/Data Science/Car price prediction capstone project/location_corrected.csv')

In [29]:
df_location.reset_index(inplace=True) # reset index


In [35]:

df_location.loc[:,['location','address','latitude','longitude']].to_csv('address.csv', index=False) # export to CSV without index column

In [36]:
df_location['country'] = df_location.address.apply(lambda x: x.split(',')[-1].strip())

In [37]:
df_location.country.value_counts()

Germany                     12627
Spain                        6473
Netherlands                  2924
Italy                        2498
France                       1469
                            ...  
4430 Ans                        1
4300 Waremme                    1
2500 Lier                       1
7740 Pecq                       1
4530 Villers-le-Bouillet        1
Name: country, Length: 314, dtype: int64

In [38]:
def getCountry(x):
    try:
        address = x

        # set up the query parameters
        params = {
            'q': address,
            'key': bingmaps_key,
        }

        # use the geocode API to get the corrected address
        response = LocationByQuery(params)

        # print the corrected address
        if response.status_code == 200:
            data = json.loads(response.response)
            try:
                country_region = data['resourceSets'][0]['resources'][0]['address']['countryRegion']
                return country_region
            except:
                return np.nan
        else:
            return np.nan
    except:
        return np.nan

In [39]:
getCountry(df_location.iloc[6301,:].address)

'Netherlands'

In [40]:
countrys = df_location[df_location['country'].str.contains('\d')].address.apply(lambda x: getCountry(x) )

In [41]:
countrys

8        Belgium
10       Belgium
13       Belgium
16       Belgium
23       Belgium
          ...   
28607    Belgium
28608    Belgium
28609    Belgium
28611    Belgium
28619    Belgium
Name: address, Length: 1503, dtype: object

In [42]:
df_location.loc[df_location['country'].str.contains('\d'), 'country'] = countrys

In [43]:
df_location.country.value_counts()

Germany              12627
Spain                 6474
Netherlands           2924
Italy                 2498
Belgium               1865
France                1469
Austria                651
Luxembourg              30
Mexico                  17
United States           10
ES                      10
Hungary                  8
Poland                   6
Esch-sur-Alzette         6
Puerto Rico              5
Berne                    4
Mons                     3
Berlaar                  3
Finland                  2
GA                       1
Peru                     1
Meeuwen-Gruitrode        1
Switzerland              1
Slovenia                 1
Tartumaa                 1
Bulgaria                 1
Czechia                  1
DE                       1
CA                       1
Portugal                 1
San Marino               1
Denmark                  1
NJ                       1
Reunion                  1
Hannut                   1
Cantabria                1
Canada                   1
N

In [45]:
df_location['country'] = df_location.address.apply(lambda x: getCountry(x))

In [46]:
df_location.to_csv('C:/Users/ihsankoo/Desktop/Clarusway/Data Science/Car price prediction capstone project/final_cleaned.csv')

In [47]:
import folium
from folium.plugins import HeatMap

In [49]:
df_location.latitude.isna().sum()

11

In [50]:
df_location.longitude.isna().sum()

11

In [51]:
df_location = df_location.dropna(subset=['latitude', 'longitude'], inplace=True)

In [58]:
df_location = pd.read_csv('C:/Users/ihsankoo/Desktop/Clarusway/Data Science/Car price prediction capstone project/final_cleaned_location.csv')

In [59]:
# Fill missing values with a default value
df_location = df_location.fillna(0)

# Create a map centered on the mean latitude and longitude
m = folium.Map(location=[df_location['latitude'].mean(), df_location['longitude'].mean()], zoom_start=2)

# Create a heatmap layer using the latitude and longitude columns
heatmap_layer = HeatMap(data=df_location[['latitude', 'longitude']], radius=15)

# Add the heatmap layer to the map
heatmap_layer.add_to(m)

# Display the map
m